<a href="https://colab.research.google.com/github/Lucky920487/Mental-Health-chatbot-Gen-AI/blob/main/Mental_Health_Chatbot_GeN_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_groq langchain_core langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)

result = llm.invoke("who is lord Ram?")
print(result.content)

Lord Rama is a major deity in Hinduism and is considered one of the most important figures in Hindu mythology. He is the seventh avatar (incarnation) of the god Vishnu and is revered as a symbol of righteousness, duty, and virtue.

According to the Hindu epic, the Ramayana, Lord Rama was a king of Ayodhya, a city in ancient India. He was born to King Dasharatha and Queen Kausalya, and was the eldest of four brothers, including Lakshmana, Bharata, and Shatrughna.

The story of Lord Rama is as follows:

* Lord Rama was exiled to the forest for 14 years by his stepmother, Queen Kaikeyi, who wanted her own son, Bharata, to become the king.
* During his exile, Lord Rama's wife, Sita, was abducted by the demon king Ravana, who took her to his kingdom in Lanka (modern-day Sri Lanka).
* Lord Rama, along with his brother Lakshmana and the monkey god Hanuman, embarked on a journey to rescue Sita and defeat Ravana.
* After a long and arduous battle, Lord Rama defeated Ravana and rescued Sita, ret

In [3]:
!pip install PyPdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 4.5 MB/s eta 0:00:00


In [6]:
!pip install chromadb

In [7]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 64.4 MB/s eta 0:00:00


In [13]:
# Changed import statement to correct the module path
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os # Import the os module
import gradio as gr

def initialize_lim():
  llm = ChatGroq(
    temperature=0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)

  return llm

def create_vector_db():
  loader = DirectoryLoader("/content/data", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
     {context}
     User: {question}
     Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = 'stuff',
      retriever = retriever,
      return_source_documents = True,
      chain_type_kwargs = {'prompt': PROMPT}
  )
  return qa_chain


print("Initializing Chatbot........")
llm = initialize_lim()

db_path = "/content/chroma_db"


if not os.path.exists(db_path):
  vector_db = create_vector_db()
else:
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma(persist_directory = db_path, embedding_function = embeddings)

qa_chain = setup_qa_chain(vector_db, llm)

def chatbot_response(user_input):
    try:
        if not user_input.strip():
            return "Please provide a valid input"

        # Use invoke instead of run and access the result properly
        response = qa_chain.invoke({"query": user_input})

        # Return just the string result for Gradio
        if isinstance(response, dict) and 'result' in response:
            return response['result']
        else:
            return str(response)

    except Exception as e:
        return f"An error occurred: {str(e)}"

with gr.Blocks(theme = 'Respair/Shiki@1.2.1') as app:
    gr.Markdown("## Mental Health Chatbot")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Type your message here...")
    clear = gr.Button("Clear")

    def respond(message, chat_history):
        bot_message = chatbot_response(message)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

app.launch()





Initializing Chatbot........


<ipython-input-13-538a60519976>:84: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dd6888feaa16a74cd9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
create_vector_db()

<ipython-input-7-68c756942ea4>:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ChromaDB created and data saved


<ipython-input-7-68c756942ea4>:27: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


In [12]:
def main():
  print("Initializing Chatbot........")
  llm = initialize_lim()

  db_path = "/content/chroma_db"


  if not os.path.exists(db_path):
    vector_db = create_vector_db()
  else:
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory = db_path, embedding_function = embeddings)

  qa_chain = setup_qa_chain(vector_db, llm)

  while True:
    query = input("\nHuman: ")
    if query.lower() == "exit":
      print("Chatbot: Take Care of yourself, Goodbye!")
      break
    # Changed from .run(query) to .invoke({"query": query}) and accessing the 'result' key
    response = qa_chain.invoke({"query": query})
    print(f"Chatbot:  {response['result']}")

if __name__ == "__main__":
  main()

Initializing Chatbot........
Chatbot:  Hello. It's nice to meet you. I see you've been reading about the concept of mental health, particularly in the context of child and adolescent mental health. The definitions you've come across from various sources, such as Rowling et al. and the World Health Organization, highlight the importance of having a sense of purpose, quality relationships, self-respect, and mastery in achieving positive mental health.

I'd like to acknowledge that these definitions are indeed incomplete, as you've noted, because they don't fully account for the impact of social and physical environments on an individual's mental well-being. It's essential to consider the complex interplay between personal factors and environmental influences when understanding mental health.

What are your thoughts on this topic? How do you think we can better understand and promote mental health, especially in children and adolescents? I'm here to listen and provide support.

Human: Exi

In [11]:
def chatbot_response(user_input):
    try:
        if not user_input.strip():
            return "Please provide a valid input"

        # Use invoke instead of run and access the result properly
        response = qa_chain.invoke({"query": user_input})

        # Return just the string result for Gradio
        if isinstance(response, dict) and 'result' in response:
            return response['result']
        else:
            return str(response)

    except Exception as e:
        return f"An error occurred: {str(e)}"

with gr.Blocks(theme = 'Respair/Shiki@1.2.1') as app:
    gr.Markdown("## Mental Health Chatbot")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Type your message here...")
    clear = gr.Button("Clear")

    def respond(message, chat_history):
        bot_message = chatbot_response(message)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

app.launch()


<ipython-input-11-90f199fa118b>:20: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3dac2d6f9db5c7929a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
